In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('', 300, 500)

# Fig 4

In [6]:
# topojson
# https://martinjc.github.io/UK-GeoJSON/json/sco/topo_lad.json simplyfiy on mapshaper.org
scot = json.loads(open("visualisation/fig4/topo_lad.json", "r").read())

In [7]:
scot['objects']['lad']['geometries'][0]

{'type': 'MultiPolygon',
 'properties': {'LAD13CD': 'E06000001',
  'LAD13CDO': '00EB',
  'LAD13NM': 'Hartlepool',
  'LAD13NMW': None},
 'id': 'E06000001',
 'arcs': [[[0, 1]],
  [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]]}

In [8]:
for i in scot["objects"]["lad"]["geometries"]:
    lad = (
        i["properties"]["LAD13NM"]
        .replace(" City", "")
        .replace("City of ", "")
        .replace(" Islands", "")
    )
    i["properties"]["LAD13NM"] = lad
    try:
        for k in df.set_index("id").columns:
            i["properties"][k] = df.set_index("id").loc[lad][k]
    except:
        print(lad)

open("visualisation/fig4/topo_lad_standard.json", "w").write(json.dumps(scot))

Hartlepool
Middlesbrough
Redcar and Cleveland
Stockton-on-Tees
Darlington
Halton
Warrington
Blackburn with Darwen
Blackpool
Kingston upon Hull, of
East Riding of Yorkshire
North East Lincolnshire
North Lincolnshire
York
Derby
Leicester
Rutland
Nottingham
Herefordshire, County of
Telford and Wrekin
Stoke-on-Trent
Bath and North East Somerset
Bristol, of
North Somerset
South Gloucestershire
Plymouth
Torbay
Bournemouth
Poole
Swindon
Peterborough
Luton
Southend-on-Sea
Thurrock
Medway
Bracknell Forest
West Berkshire
Reading
Slough
Windsor and Maidenhead
Wokingham
Milton Keynes
Brighton and Hove
Portsmouth
Southampton
Isle of Wight
County Durham
Cheshire East
Cheshire West and Chester
Shropshire
Cornwall
Isles of Scilly
Wiltshire
Bedford
Central Bedfordshire
Northumberland
Aylesbury Vale
Chiltern
South Bucks
Wycombe
Cambridge
East Cambridgeshire
Fenland
Huntingdonshire
South Cambridgeshire
Allerdale
Barrow-in-Furness
Carlisle
Copeland
Eden
South Lakeland
Amber Valley
Bolsover
Chesterfield
De

1309251

In [9]:
scot['objects']['lad']['geometries'][0]

{'type': 'MultiPolygon',
 'properties': {'LAD13CD': 'E06000001',
  'LAD13CDO': '00EB',
  'LAD13NM': 'Hartlepool',
  'LAD13NMW': None},
 'id': 'E06000001',
 'arcs': [[[0, 1]],
  [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]]}

In [93]:
ds=pd.json_normalize(scot['objects']['lad']['geometries'])

In [94]:
df=pd.read_excel('raw/EconObsData.xlsx',sheet_name='Figure4')

In [95]:
df['id']=df['lad11cd'].replace('E41000324','E09000033').replace('E41000052','E06000052').replace('E06000048','E06000057')\
    .replace('E07000100','E07000240').replace('E07000104','E07000241').replace('E07000097','E07000242').replace('E07000101','E07000243').replace('E08000020','E08000037')
df['lad11nm']=df['lad11nm'].str.replace("City of London,", "").str.replace(" City", "").str.replace("City of ", "").str.replace(" Islands", "").\
        str.replace(",", ", ").str.replace("  ", " ").str.strip()
df=pd.concat([df[df['id']=='E06000052'].replace('E06000052','E06000053'),df])
df['WFH']=df['WFH'].round(1)
df['WFH2']=df['WFH'].round(0).astype(int)

In [96]:
dk=ds.join(df.set_index('id'),on='id')
dk[dk['lad11nm'].astype(str)=='nan']

,type,id,arcs,properties.LAD13CD,properties.LAD13CDO,properties.LAD13NM,properties.LAD13NMW,lad11cd,lad11nm,WFH,_ID,WFH2
293,Polygon,E09000001,"[[2634, 2635, 2636, 2637, 2638, 2639]]",E09000001,00AA,London,None,NaN,NaN,NaN,NaN,NaN


In [97]:
open('visualisation/fig4/lad_data.json','w').write(json.dumps(list(df.T.to_dict().values())))

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


37213

In [99]:
readme = (
    "### fig4_remote_work"
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/what-does-remote-working-mean-for-regional-economies-in-the-uk/visualisation/fig4/)\n\n"
)
open("README.md", "w").write(readme)

190